## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Helper Functions

In [3]:
def filter_dataframes(path, min_date):
    '''
    Return the concated dataframe for all the files in a folder with some filters over date and columns
    
    path: folder path (str)
    min_date: minimum date (str YYYY-MM-DD)
    '''
    
    
    appended_data = []
    
    for filename in glob.glob(f'{path}*.csv'):
        
        file = filename.split('\\')[1]
        print('reading:', file)
        
        df = pd.read_csv(path + file)
        df['Data'] = pd.to_datetime(df['Data'])
        
        min_date = pd.to_datetime(min_date)
        
        df = df[df['Data'] >= min_date ].drop(columns=['index', 'state', 'station', 'station_code', 'latitude', 'longitude', 'height'])
        
        appended_data.append(df) 
        
    return pd.concat(appended_data)

## Readind Data

In [4]:
df = filter_dataframes('original_bases/', '2017-01-01')

reading: central_west.csv
reading: north.csv
reading: northeast.csv
reading: south.csv
reading: southeast.csv


In [5]:
# Treating invalid information for columns

for col in df.drop(columns=['Data', 'Hora', 'region']).columns:
    
    print(f'treating data for {col}', end=' ')
    min_value = df[col].min()
    
    df = df[df[col]!=min_value]
    
    print(f'New min value = {df[col].min()}')

treating data for PRECIPITAÇÃO TOTAL, HORÁRIO (mm) New min value = 0.0
treating data for PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB) New min value = 720.1
treating data for PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB) New min value = 720.3
treating data for PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB) New min value = 720.0
treating data for RADIACAO GLOBAL (Kj/m²) New min value = 0
treating data for TEMPERATURA DO AR - BULBO SECO, HORARIA (°C) New min value = -9.0
treating data for TEMPERATURA DO PONTO DE ORVALHO (°C) New min value = -42.5
treating data for TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C) New min value = -8.2
treating data for TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C) New min value = -8.6
treating data for TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C) New min value = -35.0
treating data for TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C) New min value = -35.0
treating data for UMIDADE REL. MAX. NA HORA ANT. (AUT) (%) New min value = 3
treating data for 

In [10]:
# Fixing Dtype for Hour column

df['Hora'] = pd.to_datetime(df['Hora']).dt.hour

In [28]:
df_left = df.drop(columns=['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']).groupby(['Data', 'Hora']).mean(numeric_only=True).reset_index()
df_right = df.groupby(['Data', 'Hora'])[['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']].sum(numeric_only=True).reset_index()

df = pd.merge(df_left, df_right, how='inner', on=['Data', 'Hora'])
del df_left, df_right

In [56]:
df.head()

,Data,Hora,"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)","PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
0,2017-01-01,0,974.566667,974.758333,973.666667,2.000000,23.675000,21.066667,24.433333,23.425000,21.616667,20.666667,86.833333,83.000000,86.083333,226.250000,5.808333,1.816667,22.0
1,2017-01-01,1,975.609091,975.663636,974.900000,1.545455,23.472727,21.136364,24.300000,23.409091,21.436364,20.818182,87.727273,82.272727,87.181818,168.181818,6.490909,2.063636,24.8
2,2017-01-01,2,960.715385,960.930769,960.276923,5.000000,22.561538,20.523077,23.269231,22.430769,20.961538,20.346154,89.307692,85.846154,88.615385,203.000000,6.161538,2.646154,11.2
3,2017-01-01,3,980.014286,980.342857,979.914286,93.571429,24.100000,21.685714,24.371429,23.885714,22.028571,21.457143,89.000000,84.571429,86.714286,243.571429,4.214286,1.757143,0.0
4,2017-01-01,4,983.512500,983.837500,983.225000,18.375000,23.662500,21.587500,24.075000,23.550000,21.950000,21.525000,89.625000,87.375000,88.625000,185.125000,3.787500,1.475000,2.6


### Acquiring target information

In [49]:
df_target = pd.read_csv('energy_demand_hourly_brazil.csv')

In [50]:
df_target['Data'] = pd.to_datetime(df_target['index'])

In [51]:
df_target['index'] = df_target['index'].apply(lambda x: x.split()[0])

In [52]:
df_target['Hora'] = df_target['Data'].dt.hour

In [53]:
df_target['Data'] = pd.to_datetime(df_target['index'])

In [54]:
df_target.drop(columns=['index'], inplace=True)

In [55]:
df_target

,hourly_demand,Data,Hora
0,34673.900,2000-01-01,0
1,33503.000,2000-01-01,1
2,32287.600,2000-01-01,2
3,31059.400,2000-01-01,3
4,30272.700,2000-01-01,4
...,...,...,...
201313,75825.344,2022-12-31,20
201314,69894.955,2022-12-31,21
201315,64612.355,2022-12-31,22
201316,61215.376,2022-12-31,23


## Joining bases

In [57]:
df_final = pd.merge(df_target, df, how='inner', on=['Data', 'Hora'])

In [58]:
df_final

,hourly_demand,Data,Hora,"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)","PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
0,54737.946,2017-01-01,0,974.566667,974.758333,973.666667,2.000000,23.675000,21.066667,24.433333,23.425000,21.616667,20.666667,86.833333,83.000000,86.083333,226.250000,5.808333,1.816667,22.0
1,54221.548,2017-01-01,1,975.609091,975.663636,974.900000,1.545455,23.472727,21.136364,24.300000,23.409091,21.436364,20.818182,87.727273,82.272727,87.181818,168.181818,6.490909,2.063636,24.8
2,53781.230,2017-01-01,2,960.715385,960.930769,960.276923,5.000000,22.561538,20.523077,23.269231,22.430769,20.961538,20.346154,89.307692,85.846154,88.615385,203.000000,6.161538,2.646154,11.2
3,52837.082,2017-01-01,3,980.014286,980.342857,979.914286,93.571429,24.100000,21.685714,24.371429,23.885714,22.028571,21.457143,89.000000,84.571429,86.714286,243.571429,4.214286,1.757143,0.0
4,52037.905,2017-01-01,4,983.512500,983.837500,983.225000,18.375000,23.662500,21.587500,24.075000,23.550000,21.950000,21.525000,89.625000,87.375000,88.625000,185.125000,3.787500,1.475000,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37938,76423.969,2021-04-30,19,962.274161,962.592282,962.177517,1159.936242,25.631208,15.582886,26.653020,25.308389,16.292282,14.809060,57.825503,51.302013,55.751678,137.580537,6.087584,2.414094,22.8
37939,74097.835,2021-04-30,20,961.382007,961.529066,961.174740,553.397924,24.396886,15.650865,25.792734,24.314187,16.158478,14.934602,61.148789,54.252595,60.269896,135.750865,5.713841,2.111073,40.2
37940,72395.729,2021-04-30,21,961.161390,961.338224,960.898842,131.610039,22.777992,15.854826,24.555985,22.731660,16.167181,15.249035,67.154440,58.938224,66.718147,138.555985,4.780309,1.620077,22.6
37941,69893.071,2021-04-30,22,965.034694,965.212245,964.671429,68.979592,23.363265,17.926531,25.095918,23.293878,18.406122,17.406122,73.816327,65.897959,72.795918,158.102041,3.981633,1.644898,19.2
